<a href="https://colab.research.google.com/github/mit1280/LLM-Agent/blob/main/Huggingface_CrewAI_Tutorial.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Install required pacakges


In [ ]:
# !pip install -q crewai-tools==0.0.15
# !pip install --q crewai
!pip install --q U crewai crewai-tools
# !pip install --q duckduckgo-search==5.0
!pip install --q langchain-huggingface langchain_community

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.5/42.5 kB 1.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 5.4 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.2/48.2 kB 3.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.9/211.9 kB 15.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 514.9/514.9 kB 35.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 131.8/131.8 kB 9.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 64.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 147.8/147.8 kB 11.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.4/211.4 kB 14.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 628.3/628.3 kB 34.

### Required Packages

In [ ]:
import os
from langchain_huggingface import HuggingFacePipeline
# from langchain.tools import DuckDuckGoSearchRun, Tool
from crewai import Agent, LLM, Task, Process, Crew
from langchain.llms import Ollama
# from langchain.tools import DuckDuckGoSearchRun
from typing import Type
from crewai.tools import BaseTool
from pydantic import BaseModel, Field

## Set huggingface env key

Create a key in huggingface. Go to https://huggingface.co/settings/tokens.

Make sure to create key with endpoint access.

In [ ]:
# from huggingface_hub import login
# login()

In [ ]:
import os
os.environ["HUGGINGFACE_API_KEY"] = "hf_xxxxxxx"

### setup agents

In [ ]:
llm = LLM(
    model="huggingface/microsoft/Phi-3.5-mini-instruct",
    temperature=0.7,
)

In [ ]:
# Agents
researcher = Agent(
  role='Boy',
  goal='Suggest a bedtime story topic quickly.',
  backstory='A curious 9-year-old who wants a bedtime story.',
  llm=llm,
  model_provider="huggingface",
  verbose=True  # Reduced verbosity for faster processing
)

analyst = Agent(
  role='Story Teller',
  goal='Create a concise bedtime story based on the topic.',
  backstory='An experienced storyteller known for captivating children.',
  llm=llm,
  model_provider="huggingface",
  verbose=True  # Reduced verbosity for faster processing
)

writer = Agent(
  role='Story Summarizer',
  goal='Summarize the story efficiently for a child.',
  backstory='A skilled writer adept at summarizing stories in a fun way.',
  llm=llm,
  model_provider="huggingface",
  verbose=True  # Reduced verbosity for faster processing
)

In [ ]:
# Tasks
research_task = Task(
  description='Quickly generate a bedtime story topic (no more than 4 words). Keep it simple and interesting for a child.',
  agent=researcher,
  expected_output='A topic in four words or less.',
  max_time=10  # Allocate up to 10 seconds for this task
)

analysis_task = Task(
  description='Create a concise bedtime story based on the topic. The story must have only five sentences, each engaging and clear.',
  agent=analyst,
  expected_output='A five-sentence bedtime story.',
  max_time=55  # Allocate up to 55 seconds for this task
)

writing_task = Task(
  description='Summarize the bedtime story into two engaging sentences suitable for a 9-year-old.',
  agent=writer,
  expected_output='A two-sentence story summary.',
  max_time=20  # Allocate up to 20 seconds for this task
)


In [ ]:
# Form the crew with a sequential process
report_crew = Crew(
  agents=[researcher, analyst, writer],
  tasks=[research_task, analysis_task, writing_task],
  process=Process.sequential
)

In [ ]:
# Execute the crew
result = report_crew.kickoff()

# Agent: Boy
## Task: Quickly generate a bedtime story topic (no more than 4 words). Keep it simple and interesting for a child.


# Agent: Boy
## Final Answer: 
Adventure in Dreamland Forest


# Agent: Story Teller
## Task: Create a concise bedtime story based on the topic. The story must have only five sentences, each engaging and clear.


# Agent: Story Teller
## Final Answer: 
Once upon a time, nestled between reality and slumber, there was a luminous forest called Dreamland. In this forest, the trees whispered secrets and the leaves shimmered with starlight. A curious young fox named Rusty ventured into the heart of Dreamland, guided by the glow of a mystical moonflower. Along his journey, he met friendly creatures and overcame gentle challenges, learning the importance of courage and kindness. As the moonflower dimmed, Rusty returned home, feeling as peaceful as the stars above, ready to embrace the sweet dreams of tomorrow.


# Agent: Story Summarizer
## Task: Summarize the bedt